In [ ]:
!pip install tensorflow-hub

In [1]:
import os
import random
import shutil
import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import sys
import tempfile
import tqdm
import time

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
movenet = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")

In [3]:
print(movenet)

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x000001AB3FD88810>


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

def preprocess_image(image_path):

    image = cv2.imread(image_path)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = tf.image.resize_with_pad(tf.convert_to_tensor(image), 192, 192)

    image = tf.cast(image, dtype=tf.uint8)

    return tf.expand_dims(image, axis=0)



In [ ]:

def extract_keypoints(image_path):

    input_image = preprocess_image(image_path)

    outputs = movenet.signatures['serving_default'](input_image)
    

    keypoints_with_scores = outputs['output_0'].numpy()[0]
    return keypoints_with_scores


In [14]:
def preprocess_image(image_path):

    image = cv2.imread(image_path)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = tf.image.resize_with_pad(tf.convert_to_tensor(image), 192, 192)

    image = tf.cast(image, dtype=tf.int32)

    return tf.expand_dims(image, axis=0)

# Extract keypoints
def extract_keypoints(image_path):
    input_image = preprocess_image(image_path)
    outputs = movenet.signatures['serving_default'](input_image)
    keypoints_with_scores = outputs['output_0'].numpy()[0]
    return keypoints_with_scores

# Test the function
keypoints = extract_keypoints("Dataset/Pose7/image6517.jpg")
print(keypoints)

[[[0.25020307 0.5547818  0.5156442 ]
  [0.24018852 0.5704604  0.81824076]
  [0.23825799 0.54271674 0.6738923 ]
  [0.2545424  0.58431816 0.51085514]
  [0.25122362 0.52353024 0.572366  ]
  [0.2999314  0.6046803  0.71790105]
  [0.3138964  0.48274928 0.8175744 ]
  [0.2382044  0.64349425 0.50724065]
  [0.3519933  0.4111061  0.7475398 ]
  [0.16276333 0.6450962  0.7645221 ]
  [0.38340908 0.31396568 0.78853285]
  [0.52587146 0.5872636  0.8392743 ]
  [0.5297412  0.51549375 0.8972923 ]
  [0.6603981  0.599906   0.90864897]
  [0.65391344 0.5154544  0.9200018 ]
  [0.77531135 0.6031971  0.8162842 ]
  [0.7684161  0.51848805 0.8861075 ]]]


In [18]:
from tqdm import tqdm

In [22]:
import os
import cv2
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

DATASET_PATH = "Dataset"  # Replace with your dataset folder path
OUTPUT_PATH = "Output_Image"  # Folder to save preprocessed images and keypoints

# Create output folders
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(f"{OUTPUT_PATH}/train", exist_ok=True)
os.makedirs(f"{OUTPUT_PATH}/val", exist_ok=True)
os.makedirs(f"{OUTPUT_PATH}/test", exist_ok=True)

# Function to normalize keypoints
def normalize_keypoints(keypoints, img_width, img_height):
    """Normalize keypoints to make them invariant to scale and translation."""
    # Center keypoints around the hips
    hips_center = (keypoints[11, :2] + keypoints[12, :2]) / 2  # LEFT_HIP and RIGHT_HIP
    keypoints[:, :2] -= hips_center  # Translate to center

    # Scale keypoints by the maximum distance from the hips center
    max_dist = max(1e-6, np.max(np.linalg.norm(keypoints[:, :2], axis=1)))
    keypoints[:, :2] /= max_dist  # Normalize by distance

    # Normalize keypoints based on image dimensions
    keypoints[:, 0] /= img_width  # x-coordinate
    keypoints[:, 1] /= img_height  # y-coordinate

    return keypoints

# Function to preprocess images and extract keypoints
def preprocess_and_extract_keypoints(image_path, save_dir):
    # Load the image
    image = cv2.imread(image_path)
    img_height, img_width, _ = image.shape
    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize and pad to 192x192
    resized_image = tf.image.resize_with_pad(tf.convert_to_tensor(image), 192, 192)
    # Convert to int32
    input_image = tf.cast(resized_image, dtype=tf.int32)
    # Add batch dimension
    input_image = tf.expand_dims(input_image, axis=0)
    
    # Perform inference with MoveNet
    outputs = movenet.signatures['serving_default'](input_image)
    keypoints_with_scores = outputs['output_0'].numpy()  # Shape: (1, 17, 3)

    # Remove the batch dimension
    keypoints_with_scores = keypoints_with_scores.squeeze()  # Shape: (17, 3)

    # Validate keypoints shape
    if keypoints_with_scores.shape != (17, 3):
        raise ValueError(f"Unexpected keypoints shape: {keypoints_with_scores.shape} for {image_path}")

    # Normalize keypoints
    normalized_keypoints = normalize_keypoints(keypoints_with_scores[:, :3], img_width, img_height)
    
    # Save preprocessed image
    preprocessed_image = tf.cast(resized_image, dtype=tf.uint8).numpy()
    image_name = os.path.basename(image_path)
    save_path = os.path.join(save_dir, image_name)
    cv2.imwrite(save_path, cv2.cvtColor(preprocessed_image, cv2.COLOR_RGB2BGR))
    
    return save_path, keypoints_with_scores, normalized_keypoints

# Gather all images and their labels
image_paths = []
labels = []

print("Gathering dataset information...")
for label in tqdm(os.listdir(DATASET_PATH), desc="Labels"):
    label_dir = os.path.join(DATASET_PATH, label)
    if os.path.isdir(label_dir):
        for image_name in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_name)
            if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(image_path)
                labels.append(label)

# Split dataset into train, validation, and test
print("Splitting dataset into train, validation, and test...")
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    test_paths, test_labels, test_size=0.5, random_state=42, stratify=test_labels
)

# Preprocess images, extract keypoints, and record annotations
def process_and_record(data_paths, data_labels, split_name):
    records = []
    print(f"Processing {split_name} data...")
    for img_path, label in tqdm(zip(data_paths, data_labels), total=len(data_paths), desc=split_name):
        save_dir = os.path.join(OUTPUT_PATH, split_name)
        save_path, keypoints, normalized_keypoints = preprocess_and_extract_keypoints(img_path, save_dir)
        records.append({
            "ImageName": os.path.basename(save_path),
            "Label": label,
            "PreprocessedPath": save_path,
            "Keypoints": keypoints.flatten().tolist(),  # Flatten raw keypoints
            "NormalizedKeypoints": normalized_keypoints.flatten().tolist()  # Flatten normalized keypoints
        })
    return records

# Process and record for each split
train_records = process_and_record(train_paths, train_labels, "train")
val_records = process_and_record(val_paths, val_labels, "val")
test_records = process_and_record(test_paths, test_labels, "test")

# Combine all records into a single DataFrame
all_records = train_records + val_records + test_records
df = pd.DataFrame(all_records)

# Save annotations to a CSV file
csv_file = f"{OUTPUT_PATH}/annotations_with_keypoints.csv"
df.to_csv(csv_file, index=False)

# Save the processed data for later use
train_file = f"{OUTPUT_PATH}/train_data.csv"
val_file = f"{OUTPUT_PATH}/val_data.csv"
test_file = f"{OUTPUT_PATH}/test_data.csv"
pd.DataFrame(train_records).to_csv(train_file, index=False)
pd.DataFrame(val_records).to_csv(val_file, index=False)
pd.DataFrame(test_records).to_csv(test_file, index=False)

print(f"Dataset preprocessing complete. Annotations saved in '{csv_file}'.")
print(f"Train data saved in '{train_file}', validation data saved in '{val_file}', and test data saved in '{test_file}'.")


Gathering dataset information...


Labels: 100%|██████████| 8/8 [00:00<00:00, 168.57it/s]


Splitting dataset into train, validation, and test...
Processing train data...


train: 100%|██████████| 6126/6126 [10:10<00:00, 10.03it/s]


Processing val data...


val: 100%|██████████| 766/766 [01:23<00:00,  9.19it/s]


Processing test data...


test: 100%|██████████| 766/766 [01:21<00:00,  9.43it/s]


Dataset preprocessing complete. Annotations saved in 'Output_Image/annotations_with_keypoints.csv'.
Train data saved in 'Output_Image/train_data.csv', validation data saved in 'Output_Image/val_data.csv', and test data saved in 'Output_Image/test_data.csv'.


In [20]:
DATASET_PATH = "Dataset"  # Replace with your dataset folder path
OUTPUT_PATH = "Output_Image"  # Folder to save preprocessed images and keypoints


# Create output folders
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(f"{OUTPUT_PATH}/train", exist_ok=True)
os.makedirs(f"{OUTPUT_PATH}/val", exist_ok=True)
os.makedirs(f"{OUTPUT_PATH}/test", exist_ok=True)

# Function to preprocess images and extract keypoints
def preprocess_and_extract_keypoints(image_path, save_dir):
    # Load the image
    image = cv2.imread(image_path)
    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize and pad to 192x192
    resized_image = tf.image.resize_with_pad(tf.convert_to_tensor(image), 192, 192)
    # Convert to int32
    input_image = tf.cast(resized_image, dtype=tf.int32)
    # Add batch dimension
    input_image = tf.expand_dims(input_image, axis=0)
    
    # Perform inference with MoveNet
    outputs = movenet.signatures['serving_default'](input_image)
    keypoints_with_scores = outputs['output_0'].numpy()[0]
    
    # Save preprocessed image
    preprocessed_image = tf.cast(resized_image, dtype=tf.uint8).numpy()
    image_name = os.path.basename(image_path)
    save_path = os.path.join(save_dir, image_name)
    cv2.imwrite(save_path, cv2.cvtColor(preprocessed_image, cv2.COLOR_RGB2BGR))
    
    return save_path, keypoints_with_scores

# Gather all images and their labels
image_paths = []
labels = []

print("Gathering dataset information...")
for label in tqdm(os.listdir(DATASET_PATH), desc="Labels"):
    label_dir = os.path.join(DATASET_PATH, label)
    if os.path.isdir(label_dir):
        for image_name in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_name)
            if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(image_path)
                labels.append(label)

# Split dataset into train, validation, and test
print("Splitting dataset into train, validation, and test...")
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    test_paths, test_labels, test_size=0.5, random_state=42, stratify=test_labels
)

# Preprocess images, extract keypoints, and record annotations
def process_and_record(data_paths, data_labels, split_name):
    records = []
    print(f"Processing {split_name} data...")
    for img_path, label in tqdm(zip(data_paths, data_labels), total=len(data_paths), desc=split_name):
        save_dir = os.path.join(OUTPUT_PATH, split_name)
        save_path, keypoints = preprocess_and_extract_keypoints(img_path, save_dir)
        records.append({
            "ImageName": os.path.basename(save_path),
            "Label": label,
            "PreprocessedPath": save_path,
            "Keypoints": keypoints.flatten().tolist()  # Flatten keypoints into a single list
        })
    return records

# Process and record for each split
train_records = process_and_record(train_paths, train_labels, "train")
val_records = process_and_record(val_paths, val_labels, "val")
test_records = process_and_record(test_paths, test_labels, "test")

# Combine all records into a single DataFrame
all_records = train_records + val_records + test_records
df = pd.DataFrame(all_records)

# Save annotations to a CSV file
csv_file = f"{OUTPUT_PATH}/annotations_with_keypoints.csv"
df.to_csv(csv_file, index=False)
print(f"Dataset preprocessing complete. Annotations saved in '{csv_file}'.")

Gathering dataset information...


Labels: 100%|██████████| 8/8 [00:00<00:00, 266.23it/s]


Splitting dataset into train, validation, and test...
Processing train data...


train: 100%|██████████| 6126/6126 [10:19<00:00,  9.89it/s]


Processing val data...


val: 100%|██████████| 766/766 [01:55<00:00,  6.65it/s]


Processing test data...


test: 100%|██████████| 766/766 [01:58<00:00,  6.47it/s]


Dataset preprocessing complete. Annotations saved in 'Output_Image/annotations_with_keypoints.csv'.


In [7]:
import pandas as pd
import numpy as np

# Load the CSV file
df = pd.read_csv("Output_Image/annotations_with_keypoints.csv")

# Extract keypoints and labels
keypoints = df["Keypoints"].apply(eval).values  # Convert string representation to list
keypoints = np.array([np.array(kp) for kp in keypoints])  # Convert to NumPy array

labels = df["Label"].values  # Extract labels


In [17]:
def normalize_keypoints(keypoints, img_width, img_height):
    """Normalize keypoints to make them translation and scale invariant."""
    # Center keypoints around the hips
    hips_center = (keypoints[11, :2] + keypoints[12, :2]) / 2  # LEFT_HIP, RIGHT_HIP
    keypoints[:, :2] -= hips_center  # Translate to center

    # Scale by the maximum distance from hips center
    max_dist = np.max(np.linalg.norm(keypoints[:, :2], axis=1))
    if max_dist > 0:
        keypoints[:, :2] /= max_dist  # Normalize by distance

    # Normalize by image dimensions for consistency
    keypoints[:, 0] /= img_width  # x-coordinate
    keypoints[:, 1] /= img_height  # y-coordinate

    return keypoints



In [8]:
keypoints

array([[0.32844234, 0.52539092, 0.60790664, ..., 0.839863  , 0.48242074,
        0.77893972],
       [0.32354698, 0.54312646, 0.39887315, ..., 0.65979719, 0.4907887 ,
        0.44088802],
       [0.34544894, 0.57157302, 0.78444451, ..., 0.98682112, 0.56880468,
        0.73097682],
       ...,
       [0.2439184 , 0.50176787, 0.63603878, ..., 0.81869018, 0.44442385,
        0.77282536],
       [0.31820512, 0.49216613, 0.78220719, ..., 0.96007848, 0.4277364 ,
        0.85164016],
       [0.42743444, 0.4626939 , 0.5809378 , ..., 0.96629739, 0.40194151,
        0.55373394]])

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)



In [24]:
# Load normalized keypoints and labels from CSV
train_data = pd.read_csv("Output_Image/train_data.csv")
val_data = pd.read_csv("Output_Image/val_data.csv")
test_data = pd.read_csv("Output_Image/test_data.csv")

# Extract features (normalized keypoints) and labels
def extract_features_and_labels(data):
    X = np.array(data["NormalizedKeypoints"].apply(eval).tolist())  # Convert string to list
    y = np.array(data["Label"])  # Labels
    return X, y

X_train, y_train = extract_features_and_labels(train_data)
X_val, y_val = extract_features_and_labels(val_data)
X_test, y_test = extract_features_and_labels(test_data)


In [25]:
# Encode class labels as integers
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

# Get the number of classes
num_classes = len(label_encoder.classes_)


In [32]:
from tensorflow import keras

# Function to convert normalized keypoints into embeddings
def landmarks_to_embedding(inputs):
    """Embed the keypoints (landmarks) into a feature vector."""
    reshaped_inputs = keras.layers.Reshape((17, 3))(inputs)  # Assuming 18 keypoints, 3 values each (x, y, score)
    flattened = keras.layers.Flatten()(reshaped_inputs)
    return flattened

# Define the model using the provided structure
def build_embedding_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)  # Update shape to 51
    embedding = landmarks_to_embedding(inputs)

    layer = keras.layers.Dense(128, activation=tf.nn.relu6)(embedding)
    layer = keras.layers.Dropout(0.5)(layer)
    layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
    layer = keras.layers.Dropout(0.5)(layer)
    outputs = keras.layers.Dense(num_classes, activation="softmax")(layer)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Build the model with updated input shape
input_shape = (51,)  # 17 keypoints * 3 (x, y, confidence)
model = build_embedding_model(input_shape, num_classes=len(label_encoder.classes_))
model.summary()



Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 51)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 17, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 51)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │         6,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,432 (60.28 KB)

 Trainable params: 15,432 (60.28 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=8,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]
)


Epoch 1/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 924us/step - accuracy: 0.2850 - loss: 1.8534 - val_accuracy: 0.3029 - val_loss: 1.8239
Epoch 2/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.2795 - loss: 1.8380 - val_accuracy: 0.2859 - val_loss: 1.8243
Epoch 3/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.2771 - loss: 1.8325 - val_accuracy: 0.3159 - val_loss: 1.7952
Epoch 4/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 763us/step - accuracy: 0.2895 - loss: 1.8208 - val_accuracy: 0.3133 - val_loss: 1.7923
Epoch 5/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 752us/step - accuracy: 0.2805 - loss: 1.8237 - val_accuracy: 0.3107 - val_loss: 1.7930
Epoch 6/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 752us/step - accuracy: 0.2824 - loss: 1.8292 - val_accuracy: 0.3146 - val_loss: 1.7970
Epoch 7/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 744us/step - accuracy: 0.2806 - loss: 1.8308 - val_accuracy: 0.3055 - val_loss: 1.7978
Epoch 8/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 751us/step - accuracy: 0.2874 - loss: 1

In [15]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32
)


Epoch 1/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5067 - loss: 1.3325 - val_accuracy: 0.6142 - val_loss: 1.1272
Epoch 2/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - accuracy: 0.5139 - loss: 1.2704 - val_accuracy: 0.6134 - val_loss: 1.1079
Epoch 3/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - accuracy: 0.5391 - loss: 1.2676 - val_accuracy: 0.6052 - val_loss: 1.0812
Epoch 4/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5120 - loss: 1.2608 - val_accuracy: 0.6109 - val_loss: 1.0896
Epoch 5/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step - accuracy: 0.5155 - loss: 1.2810 - val_accuracy: 0.6093 - val_loss: 1.0829
Epoch 6/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - accuracy: 0.5252 - loss: 1.2726 - val_accuracy: 0.6158 - val_loss: 1.0596
Epoch 7/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - accuracy: 0.5293 - loss: 1.2562 - val_accuracy: 0.6126 - val_loss: 1.0676
Epoch 8/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - accuracy: 0.5185 - loss: 1.260